<a href="https://colab.research.google.com/github/MengyuLIANG1/QRM_works/blob/main/QRM_EX1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Import libraries
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
from scipy.stats import norm

In [ ]:
# load data
yf.pdr_override() # Set Yahoo Finance API for pandas_datareader

start_date = "1993-04-06"
end_date = "2023-04-06"

# Load S&P 500 data from Yahoo Finance
sp500 = pdr.get_data_yahoo("^GSPC", start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [ ]:
sp500

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-04-06,442.290009,443.380005,439.480011,441.160004,441.160004,293680000
1993-04-07,441.160004,442.730011,440.500000,442.730011,442.730011,300000000
1993-04-08,442.709991,443.769989,440.019989,441.839996,441.839996,284370000
1993-04-12,441.839996,448.369995,441.839996,448.369995,448.369995,259690000
1993-04-13,448.410004,450.399994,447.660004,449.220001,449.220001,286690000
...,...,...,...,...,...,...
2023-03-30,4046.739990,4057.850098,4032.100098,4050.830078,4050.830078,3930860000
2023-03-31,4056.179932,4110.750000,4056.179932,4109.310059,4109.310059,4525120000
2023-04-03,4102.200195,4127.660156,4098.790039,4124.509766,4124.509766,4234700000


### 2. Compute the average $\mu$ and the variance $\sigma^2$ of the losses (-log-return):

$$L_t=-(logP_t -logP_{t-1})$$

In [ ]:
#Calculate daily log-returns and losses
prices=sp500['Adj Close']
log_returns = np.log(prices) - np.log(prices.shift(1))
losses = -log_returns.dropna()

In [ ]:
losses

Date
1993-04-07   -0.003552
1993-04-08    0.002012
1993-04-12   -0.014671
1993-04-13   -0.001894
1993-04-14    0.001247
                ...   
2023-03-30   -0.005699
2023-03-31   -0.014333
2023-04-03   -0.003692
2023-04-04    0.005814
2023-04-05    0.002495
Name: Adj Close, Length: 7554, dtype: float64

In [ ]:
# Calculate the  the average 𝜇 and the variance 𝜎2 of the losses
mu = losses.mean()
sigma2 = losses.var()
print('Average is : {}'.format(mu))
print('Variance is : {}'.format(sigma2))

Average is : -0.00029480877625998156
Variance is : 0.0001398701996109016


### 3. Compute empirically the $VaR_{\alpha}$ and the $ES_{\alpha}$from the data with $\alpha$ = 99%

In [ ]:
# Step 4: Compute empirical VaR and ES
alpha = 0.99
var_empirical = np.percentile(losses, alpha*100)
es_empirical = losses[losses >= var_empirical].mean()
print("Empirical VaR (99%):", var_empirical)
print("Empirical ES (99%):", es_empirical)

Empirical VaR (99%): 0.03343896081420468
Empirical ES (99%): 0.04892797623674416


### 4. Compare these values to the same computed using the Gaussian approximation

In [ ]:
# Step 5: Compute Gaussian VaR and ES
z_alpha = norm.ppf(alpha)
var_gaussian = mu + sigma2**0.5 * z_alpha
es_gaussian = mu + sigma2**0.5 * norm.pdf(z_alpha) / (1 - alpha)
print("Gaussian VaR (99%):", var_gaussian)
print("Gaussian ES (99%):", es_gaussian)

Gaussian VaR (99%): 0.027218147341625594
Gaussian ES (99%): 0.031225808886896723


### 5. Compute the difference between the empirical risk measures and the one computed from the Gaussian model

In [ ]:
var_diff = var_empirical -var_gaussian
es_diff = es_empirical-es_gaussian
print('VaR difference:', var_diff)
print('ES difference:', es_diff)

VaR difference: 0.006220813472579086
ES difference: 0.017702167349847434


### 6. Compute the percentage increase in the risk measures from the empirical to the Gaussian ones

In [ ]:
var_increase = (-var_diff) / var_empirical * 100
es_increase = (-es_diff) / es_empirical * 100
print('VaR percentage increase: {:.2f}'.format(var_increase), '%')
print('ES percentage increase: {:.2f}'.format(es_increase), '%')


VaR percentage increase: -18.60 %
ES percentage increase: -36.18 %


### 7. Find the largest loss in the database $X_N$

In [ ]:
largest_loss=losses.max()
print('Largest loss:',largest_loss)

Largest loss: 0.12765219747281709


### 8. Find the second largest loss in the database $X_{N-1}$

In [ ]:
second_largest_loss = losses.nlargest(2).iloc[-1]
print('Second largest loss:', second_largest_loss)

Second largest loss: 0.09994485240247286


### 9. Compute the probability of find a loss bigger than $X_{N-1}$ in the Gaussian model

In [ ]:
p_loss_greater_than_second_largest = 1 - norm.cdf(second_largest_loss, mu, sigma2**0.5)

print('Probability of finding a loss greater than the second largest loss in the Gaussian model:', p_loss_greater_than_second_largest)

Probability of finding a loss greater than the second largest loss in the Gaussian model: 0.0
